# GradCam: Visual Explanations via Gradient-based Localization and other visualization techniques.

At the end of this lab, you will get familiarized with

*    PCA, T-SNE and Umap visualization of layer results
*   Applying GradCam for a CNN classification
*   Visualizing gradcam results

**Remember this is a graded exercise.**

*   For every plot, make sure you provide appropriate titles, axis labels, legends, wherever applicable.
*   Create reusable functions where ever possible, so that the code could be reused at different places.
*   Add sufficient comments and explanations wherever necessary.
*   **Once you have the code completed, use GPU to train model faster.**
* This lab must be worked in pairs. Make sure that when you submit your lab you change the name to: **05_lab_Student1FullName_Student2FullName.ipynb**
---

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Exercise 1: Features extraction (1.5 points)

*In this section, we explored different techniques available to visualize via dimensionality reduction features developed through CNN models, first we train a model and extract its features. We use colorectal_histology from tfds leveraged before. Remember that this is multiclass problem with 8 possible categories.*


1**.1 First, lets build a model and assess its performance:**
*   Load "colorectal_histology" from TFDS
*   Only keep a 20% of the dataset as test set, as we will not be training the model.
*   Convert the labels to one-hot encoded form.
*   Normalize the images.

**1.2 Load a fine tune a MobileNet model pretrained on ImageNet**


*   Load the model `05_lab_gradcam_model_exercise1_50epochs.keras` provided to you. This was trained for 50 epochs on this dataset, starting from the Imagenet pretrained MobileNet.



In [ ]:
#solution

**1.3 Build a model to extract the features**

*   Obtain the name of the layers. Look for *global_max_pooling2d* layer, as this will be the one where we extract the features from.
*   Create a model that outputs the features from the chosen layer. Extract the intermediate features by using the predict function from the model with the test set. **Hint:** tensorflow.keras.models has a Model function that you can use to create the intermediate model. Explore its parameters






In [ ]:
#solution

## Exercise 2: Feature visualization with PCA (1 point)

*Principal component analysis is a popular technique for analyzing large datasets containing a high number of dimensions by reducing dimensionality while preserving the maximum amount of information, and enabling the visualization of multidimensional data.*

*For the features extracted, apply PCA and visualize the results. Comment on what you see and the advantages of this visualization in terms of explainability.*

**2.1 Perform and visualize PCA:**

*   Flatten the feature maps in order to use them in the PCA. You can do this with the reshape function.
*   Perform PCA from sklearn.decomposition with 2 components. You can use sklearn function for this.
*   Plot the resulting 2 components, color the observations by class.



In [ ]:
# solution

## Exercise 3: Feature visualization with T-SNE (1 point)

*In order to visualize the features of a higher dimension data, t-SNE can be used. t-SNE converts the affinities of the data points to probabilities. It recreates the probability distribution in a low-dimensional space. It is very helpful in visualizing features of different layers in a neural network.*

*You can find more information about t-SNE [here](https://scikit-learn.org/stable/modules/manifold.html#t-distributed-stochastic-neighbor-embedding-t-sne)*

**3.1 Use TSNE to visualize the features extracted from the neural network before. Obtain 2 components so you can easily visualize it in a 2-D plot, show your results**
**Hint:** TSNE function is available in the *sklearn.manifold* package.


In [ ]:
# solution

# Exercise 4: Visualize features with UMAP (1 point)

*In this section, we will visualize different layers results via the dimensionality reduction technique UMAP (Uniform Manifold Approximation and Projection). This technique is a non-linear dimensionality reduction algorithm that preserves both local and global structures in the data. You can read more on its [paper](https://arxiv.org/abs/1802.03426)*



**4.1 Use UMAP to visualize results of last layer for the test set. Follow this steps:**
*   Apply the dimensionality reduction with umap from umap.umap_.
*   Set the model to use 5 neighbors and a minimum distance of 0.1 with euclidean as distance metric.
*   Once fitted and transform, visualize the UMAP embeddings as a scatterplot, with the classes as color. Use cmap = "tab10" inside the plt.scatter.



In [ ]:
# solution

# Exercise 5: Applying GradCam with a pre-trained model (1.5 points)

 Gradient-weighted Class Activation Mapping (Grad-CAM) is a technique to visualize CNN's class decisions. It shows a visual explanation to make a more transparent models. It allows to see where the layer is focusing on to pick a class. You can learn more about GradCam from a code perspective on [keras tutorial](https://keras.io/examples/vision/grad_cam/) and theoretical aspects from the [paper](https://arxiv.org/abs/1610.02391)

**5.1 Using a VGG16 already trained, you will apply gradcam to 5 ImageNet images provided in the folder "data_imagenet_sample".**

*   Load the VGG16. model pretrained in ImageNet. Make sure to set include_top = True in this case as we are going to use ImageNet images.
*   Look for the name of the last convolution layer. Save the name in order to plot its gradcam visualization



In [ ]:
from keras.backend import clear_session
clear_session()

In [ ]:
# solution

**5.2 Explore the 5 imagenet images:**

*   Load the images provided in `data_imagenet_sample`
*   Preprocessed them, according to what the model requires. Hint: look for the preprocess function in keras.applications.vgg16 (starter code provided)
*   Predict the class for each image. Obtain the probability and label. For this, keras.applications.imagenet_utils provides a decode predictions function. A starter code for the function to do this is provided.
*   Plot the image and print both the probability and label


You can use this base code to load the images:
```python
from keras.applications.vgg16 import VGG16, preprocess_input
# Load and preprocess the ImageNet images
def load_images(folder_path, images_names):
  image_size = (224, 224)
  images = []
  # Load images
  for image_name in images_names:
    # Load
    img = image.load_img(folder_path + image_name, target_size=image_size)
    # Preprocess image
    preprocessed_img = preprocess_input(image.img_to_array(img))
    # Add to images
    images.append(preprocessed_img)
  return images

# Prediction function
def pred_model(input_batch, model):
  # Remove last layer's softmax
  model.layers[-1].activation = None
  # Get predictions for input_batch: your code should go below
  # Get decoded predictions: your code should go below
  return decoded_predictions

```


In [ ]:
# solution

**5.3 Create the gradcam heatmap:**
*   The first step is to create a new model that maps the input image to the activations of the last convolutional layer and the output predictions.
*   Use a GradientTape to compute the gradient of the top predicted class (or a chosen class) with respect to the activations of the last convolutional layer. Use the model created as first step for this.
*   Calculate the relevance scores by taking the gradient of the output neuron (top predicted or chosen) with respect to the output feature map of the last convolutional layer.
*   Calculate a vector where each entry is the mean intensity of the gradient over a specific feature map channel. Hint: use tf.reduce_mean
*   Multiply each channel in the feature map array by its corresponding pooled gradient value to obtain the heatmap class activation.
*   For visualization purposes, normalize the heatmap values between 0 and 1. Hint: Use the functions tf.maximum and tf.math.reduce_max
* Make sure the final heatmap is a NumPy array that you can display with plt.matshow


In [ ]:
# solution
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
  # Your code here
  return heatmap

**5.4 Display the heatmap for the 5 images**

In [ ]:
# solution

**5.5 Display a superimposed visualization of the 5 images:**

Create a function that does the following:
*   Rescale the heatmap values to the range of 0-255 and use the "jet" colormap to colorize the heatmap. Use the cm.get_cmap function
*   Convert the colorized heatmap to a PIL (Python Imaging Library) image, resize it to the dimensions of the original input image, and convert it back to a NumPy array.
*   Blend the colorized heatmap with the original image using the specified alpha value. Alpha should be a value that can be determined to set the transparency of the heatmap. This looks something like this: jet_heatmap * alpha + img. Then apply keras.utils.array_to_img to the result. This will create the superimposed image to be plotted with plt.imshow



In [ ]:
# solution
def display_gradcam(img, heatmap, alpha=0.4):
  # Your code here
  return superimposed_img


# Exercise 6: Apply GradCam to with Chest X-ray images dataset (3 points)

*You have used this dataset in the last lab, it contains 5,863 Chest X-Ray images classified into 2 categories (Pneumonia/Normal). The normal chest X-ray shows clear lungs without any areas of abnormal opacification in the image. Bacterial pneumonia typically exhibits a focal lobar consolidation, whereas viral pneumonia manifests with a more diffuse ‘‘interstitial’’ pattern in both lungs.*



A code to load the data and print a couple of images is provided.

In [ ]:
from keras.backend import clear_session
clear_session()
from tensorflow.keras.preprocessing.image import ImageDataGenerator
input_shape = (128, 128, 3)
batch_size = 32
n_classes = 2

folder_path_ex6 = "your/data/path"

# Define the paths to your train, test, and validation directories
train_dir = folder_path_ex6 + '/train'
test_dir = folder_path_ex6 + '/test'
val_dir = folder_path_ex6 + '/val'

# Define the parameters for data augmentation and normalization
batch_size = 64
image_size = (128, 128)

# Create an instance of the ImageDataGenerator and set the preprocessing options
datagen = ImageDataGenerator(rescale=1.0/255)

# Load and preprocess the training data
train_ds = datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

# Load and preprocess the test data
test_ds = datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

# Load and preprocess the validation data
valid_ds = datagen.flow_from_directory(
    val_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

# Get the class indices from the flow_from_directory output
class_indices = train_ds.class_indices

# Class names
class_names = {v: k for k, v in class_indices.items()}


In [ ]:
# Printing 5 images
n_samples = 5
idx = {}

# Getting index per each class
for number in range(n_classes):
    indices = np.where(train_ds.classes == number)[0][:n_samples]
    idx[number] = indices

# Plotting
fig, axes = plt.subplots(n_samples, n_classes, figsize=(10, 10))

for class_i, indices in idx.items():
    for id, idx in enumerate(indices):
        batch_x, batch_y = train_ds[idx // batch_size]
        image = batch_x[idx % batch_size]
        axes[id, class_i].imshow(image, cmap=plt.get_cmap('gray'))
        axes[id, class_i].set_title(f'Image {id+1}, class {class_i}', fontsize=5)
        axes[id, class_i].axis('off')

plt.tight_layout()
plt.show()


**2.2 Apply GradCam to train models. For this consider:**
*  We are providing you with 2 models `05_lab_gradcam_model_10epochs.keras` and `05_lab_gradcam_model_25epochs.keras`. One was trained for 10 epochs and the other one for 25 epochs with a ResNet50. Load them into your environment.



In [ ]:
#solution

**2.3 Obtain your gradcam visualization for `both` models:**
* Obtain the gradcam superimposed visualization for 5 images. Choose the last convolutional layer to do this.
* Also print the actual and predicted class
* Comment on the results - which one you think is making more sense?


In [ ]:
#solution

Comment on the differences:

**SOLUTION**



---

# **End of Lab 04: Interpretation with GradCAM**